# Hands on Session Langchain  LLM Chatbot Development:
Research Telegram  using OpenAI Chatbot
Hands-on Session

In [ ]:
#Install python requirements
#!pip install openai, langchain, telebot

In [3]:
#Initiate required library
import openai
import os
import pickle

from langchain.chat_models import AzureChatOpenAI
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.agents import Tool
from langchain.agents import AgentExecutor
from langchain.utilities import BingSearchAPIWrapper
from langchain.agents.conversational_chat.base import ConversationalChatAgent
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate

In [4]:
#Load environment and credential
from dotenv import load_dotenv
load_dotenv('.env')

os.environ["OPENAI_API_KEY"] = os.getenv('AZURE_OPENAI_API_KEY')
os.environ["BING_SUBSCRIPTION_KEY"] = os.getenv('BING_SUBSCRIPTION_KEY')
os.environ["BING_SEARCH_URL"] = os.getenv('BING_SEARCH_URL')
os.environ["TELEGRAM_BOT_KEY"] = os.getenv('TELEGRAM_BOT_KEY')

openai.api_type = "azure"
openai.api_key = os.getenv('AZURE_OPENAI_API_KEY')
openai.api_base = os.getenv("RESOURCE_ENDPOINT")


In [ ]:
#Create embedding, split character and minimize the size to the first 10 document in the paper PDF

#from langchain.embeddings.openai import OpenAIEmbeddings
#from langchain.document_loaders import UnstructuredPDFLoader
#from langchain.document_loaders import OnlinePDFLoader
#from langchain.text_splitter import RecursiveCharacterTextSplitter

#loader = OnlinePDFLoader("https://arxiv.org/pdf/2304.08485.pdf", )
#loader = UnstructuredPDFLoader("docs/2303.18223.pdf")
#docs = loader.load()
#text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
#docs = text_splitter.split_documents(docs)
#docs = docs[0:100]

In [5]:
from langchain.embeddings.openai import OpenAIEmbeddings

In [6]:
#Initiate OpenAI Embeddings (cost incurred)
embeddings = OpenAIEmbeddings(
   deployment='Embedding_ada_002',
  openai_api_version=os.getenv("API_VERSION_ADA"),
  openai_api_base=os.getenv("RESOURCE_ENDPOINT_ADA"),
  openai_api_type="azure",
  openai_proxy=None,
  embedding_ctx_length=8191,
  openai_api_key=os.getenv('AZURE_OPENAI_API_KEY_ADA'),
  chunk_size=1)

In [ ]:
#Save paper embedding
#db = FAISS.from_documents(docs, embeddings)
#with open('paper_embedding.p', 'wb') as f:
    #dump information to that file
    #pickle.dump(db, f)

In [7]:
#Load paper embedding
with open('paper_embedding.p', 'rb') as f:
    db = pickle.load(f)
    db.embedding_function = embeddings.embed_query

In [8]:
#initiate LLM, QA Retriever and BingSearch API Tools
llm = AzureChatOpenAI(deployment_name="ChatGPT_DSC_2", openai_api_version="2023-08-01-preview",
                  temperature=0, openai_api_base=openai.api_base, verbose=True)
retriever = db.as_retriever()
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
search = BingSearchAPIWrapper()

In [9]:
#Initiate tools consisting of Retrieval QA and BingSearch
tools = [
        Tool(
            name="Paper QA",
            func=qa.run,
            description="Useful when human have questions about 'the paper' or mention 'the paper' in the input. Any question regarding paper will use this tools. Action input is human question.",
        ),
        Tool(name="Internet Search",
            func=search.run,
            description="Only useful when human ask you to search internet and search regarding research. Only use tool when the human mention 'internet'",
        )
]

In [10]:
#Initiate simple prompt template
template = """You are a chatbot helping human to research paper using the "Paper QA" tools.
TOOLS:
-------------
You have access to the following tools:

{chat_history}
Human: {input}
Chatbot:
{agent_scratchpad}"""

prompt = PromptTemplate(
    input_variables=["chat_history", "input","agent_scratchpad"], template=template
)

In [11]:
#Initiate memory, agent and agent chain to converse with the chatbot
memory = ConversationBufferWindowMemory(memory_key="chat_history", k=3, return_messages=True)
agent = ConversationalChatAgent.from_llm_and_tools(llm=llm, prompt=prompt, tools=tools, memory=memory, verbose=True)
agent_chain = AgentExecutor(agent=agent, tools=tools, llm=llm, verbose=True, memory=memory)

In [12]:
#Example questions
agent_chain.run(input="Who are the authors of the paper?")



> Entering new AgentExecutor chain...
{
    "action": "Paper QA",
    "action_input": "Who are the authors of the paper?"
}
Observation: The authors of the paper are Wayne Xin Zhao, Kun Zhou, Junyi Li, Tianyi Tang, Xiaolei Wang, Yupeng Hou, Yingqian Min, Beichen Zhang, Junjie Zhang, Zican Dong, Yifan Du, Chen Yang, Yushuo Chen, Zhipeng Chen, Jinhao Jiang, Ruiyang Ren, Yifan Li, Xinyu Tang, Zikang Liu, Peiyu Liu, Jian-Yun Nie, and Ji-Rong Wen.
Thought:{
    "action": "Final Answer",
    "action_input": "The authors of the paper are Wayne Xin Zhao, Kun Zhou, Junyi Li, Tianyi Tang, Xiaolei Wang, Yupeng Hou, Yingqian Min, Beichen Zhang, Junjie Zhang, Zican Dong, Yifan Du, Chen Yang, Yushuo Chen, Zhipeng Chen, Jinhao Jiang, Ruiyang Ren, Yifan Li, Xinyu Tang, Zikang Liu, Peiyu Liu, Jian-Yun Nie, and Ji-Rong Wen."
}

> Finished chain.


'The authors of the paper are Wayne Xin Zhao, Kun Zhou, Junyi Li, Tianyi Tang, Xiaolei Wang, Yupeng Hou, Yingqian Min, Beichen Zhang, Junjie Zhang, Zican Dong, Yifan Du, Chen Yang, Yushuo Chen, Zhipeng Chen, Jinhao Jiang, Ruiyang Ren, Yifan Li, Xinyu Tang, Zikang Liu, Peiyu Liu, Jian-Yun Nie, and Ji-Rong Wen.'

In [ ]:
agent_chain.run(input="Summarize the paper for me")

In [ ]:
agent_chain.run(input="Search for LLM or Large Language Model in the internet")

In [ ]:
# Initiate telegram bot instance using Telebot
# Need to talk to @BotFather on Telegram to create Bot API.
# In this hands on session there is one instance already created @askthepaperbot, so no need to run this part
# After that you can run the agent and talk with the ResearchGPT Chatbot in Telegram

import telebot

bot = telebot.TeleBot(os.getenv('TELEGRAM_BOT_KEY'))

def generate_answer(text):
    try:
        result = agent_chain.run(input=text)


    except Exception as e:
        return f"Maaf!! Ada error terkait: {e}"

    return result


@bot.message_handler(content_types=['text'])
def send_text(message):
    answer = generate_answer(message.text)
    bot.send_message(message.chat.id, answer)

bot.polling()